In [1]:
import netket as nk
import numpy as np
from netket import experimental as nkx
from deepnets.net.ConvNext import ConvNext
from deepnets.net import ResCNN
from deepnets.net.ConvNext.heads import OutputHead
from deepnets.system import Shastry_Sutherland
from deepnets.system import Square_Heisenberg
from netket.nn.blocks import SymmExpSum
from deepnets.nn.blocks import FlipExpSum
from deepnets.net.ConvNext.encoder import Encoder
import jax
import flax
import flax.linen as nn

/Users/rajah.nutakki/uv_envs/nk_pro/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#System
L = 6
J = [0.8,1.0]
system = Square_Heisenberg(L = L, J = J)
#ConvNext
n_blocks = (4,)
features = (12,)
expansion_factor = 2
downsample_factor = 2 #linear_patch_size 
kernel_size = (2,2)
output_depth = 1
final_features = features[0]
network = ConvNext(lattice_shape = (L,L), n_blocks = n_blocks, features = features, expansion_factor = expansion_factor, Head = OutputHead, kernel_size = kernel_size, downsample_factor = downsample_factor, final_features = final_features, extract_patches = system.extract_patches_as2d,Encoder=Encoder)

/Users/rajah.nutakki/repos/netket_pro/deepnets/system/system.py:162: UserWarning: len(sign_rule) and len(J) mismatched, increasing length of sign_rule by repeating first value...
  warnings.warn(


In [29]:
vars = network.init(jax.random.PRNGKey(0),np.ones((1,L**2)))

In [46]:
tabulate_fn = nn.tabulate(
    network, jax.random.key(0), compute_flops=True, compute_vjp_flops=True, table_kwargs={'width':100})

In [47]:
print(tabulate_fn(np.ones((1, L**2))))


                                          ConvNext Summary                      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━
┃ path          ┃ module        ┃ inputs        ┃ outputs      ┃ flops ┃ vjp_flo
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━
│               │ ConvNext      │ float64[1,36] │ complex128[… │ 64684 │ 193411 
├───────────────┼───────────────┼───────────────┼──────────────┼───────┼────────
│ stem          │ PatchStem     │ float64[1,36] │ float64[1,3… │ 972   │ 2856   
├───────────────┼───────────────┼───────────────┼──────────────┼───────┼────────
│ stem/Dense_0  │ Dense         │ float64[1,3,… │ float64[1,3… │ 972   │ 2856   
│               │               │               │              │       │        
│               │               │               │              │       │        
│               │               │               │              │       │        
│               │          

In [49]:
depth = 4
features = 12
kernel_width=2
system = Square_Heisenberg(L = 3, J = [1.0], sz_sector=1/2)
net_CNN = ResCNN(depth,features,kernel_width,system)

In [50]:
tabulate_fn = nn.tabulate(
    net_CNN.network, jax.random.key(0), compute_flops=True, compute_vjp_flops=True, table_kwargs={'width':100})

In [52]:
import rich
print(tabulate_fn(np.ones((1, 3**2))))


                                          ConvReLU Summary                      
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━
┃ path      ┃ module    ┃ inputs         ┃ outputs         ┃ flops    ┃ vjp_flop
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━
│           │ ConvReLU  │ float64[1,9]   │ complex128[1]   │ 52069484 │ 52144400
├───────────┼───────────┼────────────────┼─────────────────┼──────────┼─────────
│ layers_0  │ Conv      │ float64[1,3,3… │ float64[1,3,3,… │ 972      │ 2877    
│           │           │                │                 │          │         
│           │           │                │                 │          │         
│           │           │                │                 │          │         
│           │           │                │                 │          │         
│           │           │                │                 │          │         
├───────────┼───────────┼──

In [53]:
52069484/64684

804.9824376971121